<a href="https://colab.research.google.com/github/krishna324-art/RAG---Vanilla/blob/main/chunking_strategies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    #!pip install -U torch # requires torch 2.1.1+ (for efficient sdpa implementation)
    !pip install PyMuPDF # for reading PDFs with Python
    !pip install tqdm # for progress bars
   # !pip install sentence-transformers # for embedding models
    !pip install accelerate # for quantization model loading
    !pip install bitsandbytes # for quantizing models (less storage space)
    !pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference

[INFO] Running in Google Colab, installing requirements.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 63.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for flash-attn
  Running setup.py clean for flash-attn
Failed to build flash-attn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (flash-attn)


In [ ]:
!pip install flash-attn --no-build-isolation

  Using cached flash_attn-2.8.3.tar.gz (8.4 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for flash-attn
  Running setup.py clean for flash-attn
Failed to build flash-attn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (flash-attn)


In [ ]:
# Download PDF file
import os
import requests

# Get PDF document
pdf_path = "human-nutrition-text.pdf"

# Download PDF if it doesn't already exist
if not os.path.exists(pdf_path):
  print("File doesn't exist, downloading...")

  # The URL of the PDF you want to download
  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

  # The local filename to save the downloaded file
  filename = pdf_path

  # Send a GET request to the URL
  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
      # Open a file in binary write mode and save the content to it
      with open(filename, "wb") as file:
          file.write(response.content)
      print(f"The file has been downloaded and saved as {filename}")
  else:
      print(f"Failed to download the file. Status code: {response.status_code}")
else:
  print(f"File {pdf_path} exists.")

File human-nutrition-text.pdf exists.


In [ ]:
import fitz # (pymupdf, found this is better than pypdf for our use case, note: licence is AGPL-3.0, keep that in mind if you want to use any code commercially)
from tqdm.auto import tqdm # for progress bars, requires !pip install tqdm

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip() # note: this might be different for each doc (best to experiment)

    # Other potential text formatting functions can go here
    return cleaned_text

# Open PDF and get lines/pages
# Note: this only focuses on text, rather than images/figures etc
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.

    Parameters:
        pdf_path (str): The file path to the PDF document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text()  # get plain text encoded as UTF-8
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number - 41,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''}]

In [ ]:
def chunk_text(text: str, chunk_size: int = 500) -> list:
    """
    Splits text into chunks of approx. `chunk_size` characters.
    """
    chunks = []
    current_chunk = ''
    words = text.split()

    for word in words:
        # Check if adding the word exceeds chunk size
        if len(current_chunk) + len(word) + 1 <= chunk_size:
            current_chunk += (word + ' ')
        else:
            # Store current chunk and start new one
            chunks.append(current_chunk.strip())
            current_chunk = word + ' '

    # Add the last chunk if not empty
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


def chunk_pdf_pages(pages_and_texts: list, chunk_size: int = 500) -> list[dict]:
    """
    Takes PDF pages with text and splits them into chunks.

    Returns a list of dicts with page_number, chunk_index, and chunk_text.
    """
    all_chunks = []
    for page in pages_and_texts:
        page_number = page["page_number"]
        page_text = page["text"]

        chunks = chunk_text(page_text, chunk_size=chunk_size)
        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "page_number": page_number,
                "chunk_index": i,
                "chunk_char_count": len(chunk),
                "chunk_word_count": len(chunk.split()),
                "chunk_token_count": len(chunk) / 4,  # rough token estimate
                "chunk_text": chunk
            })
    return all_chunks


# Example usage
chunked_pages = chunk_pdf_pages(pages_and_texts, chunk_size=500)
print(f"Total chunks: {len(chunked_pages)}")
print(f"First chunk (page {chunked_pages[0]['page_number']}): {chunked_pages[0]['chunk_text'][:200]}...")


Total chunks: 3321
First chunk (page -41): Human Nutrition: 2020 Edition...


In [ ]:
import random, textwrap

# ---------- Sampling & Pretty Printing ----------
def _scattered_indices(n: int, k: int, jitter_frac: float = 0.08) -> list[int]:
    """Evenly spaced anchors + random jitter → indices scattered across [0, n-1]."""
    if k <= 0:
        return []
    if k == 1:
        return [random.randrange(n)]
    anchors = [int(round(i * (n - 1) / (k - 1))) for i in range(k)]
    out, seen = [], set()
    radius = max(1, int(n * jitter_frac))
    for a in anchors:
        lo, hi = max(0, a - radius), min(n - 1, a + radius)
        j = random.randint(lo, hi)
        if j not in seen:
            out.append(j); seen.add(j)
    while len(out) < k:
        r = random.randrange(n)
        if r not in seen:
            out.append(r); seen.add(r)
    return out

def _draw_boxed_chunk(c: dict, wrap_at: int = 96) -> str:
    header = (
        f" Chunk p{c['page_number']} · idx {c['chunk_index']}  |  "
        f"chars {c['chunk_char_count']} · words {c['chunk_word_count']} · ~tokens {c['chunk_token_count']} "
    )
    # Wrap body text, avoid breaking long words awkwardly
    wrapped_lines = textwrap.wrap(
        c["chunk_text"], width=wrap_at, break_long_words=False, replace_whitespace=False
    )
    content_width = max([0, *map(len, wrapped_lines)])
    box_width = max(len(header), content_width + 2)  # +2 for side padding

    top    = "╔" + "═" * box_width + "╗"
    hline  = "║" + header.ljust(box_width) + "║"
    sep    = "╟" + "─" * box_width + "╢"
    body   = "\n".join("║ " + line.ljust(box_width - 2) + " ║" for line in wrapped_lines) or \
             ("║ " + "".ljust(box_width - 2) + " ║")
    bottom = "╚" + "═" * box_width + "╝"
    return "\n".join([top, hline, sep, body, bottom])

def show_random_chunks(pages_and_texts: list, chunk_size: int = 500, k: int = 5, seed: int | None = 42):
    if seed is not None:
        random.seed(seed)
    all_chunks = chunk_pdf_pages(pages_and_texts, chunk_size=chunk_size)
    if not all_chunks:
        print("No chunks to display.");
        return
    idxs = _scattered_indices(len(all_chunks), k)
    print(f"Showing {len(idxs)} scattered random chunks out of {len(all_chunks)} total:\n")
    for i, idx in enumerate(idxs, 1):
        print(f"#{i}")
        print(_draw_boxed_chunk(all_chunks[idx]))
        print()

# ---------- Run ----------
assert 'pages_and_texts' in globals(), "Run: pages_and_texts = open_and_read_pdf(pdf_path) first."
show_random_chunks(pages_and_texts, chunk_size=500, k=5, seed=42)

Showing 5 scattered random chunks out of 3321 total:

#1
╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║ Chunk p-9 · idx 0  |  chars 290 · words 49 · ~tokens 72.5                                        ║
╟──────────────────────────────────────────────────────────────────────────────────────────────────╢
║ Skylar Hara Skylar Hara is an undergraduate student student in the Tropical Agriculture and the  ║
║ Environment program at the University of Hawai‘i at Mānoa. She has a growing love for plants and ║
║ hopes to go to graduate school to conduct research in the future. About the Contributors |       ║
║ xxxiii                                                                                           ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝

#2
╔═════════════════════════════════════════════════════════════════════════════════════════════════╗
║ Chunk p198 · idx 0  |  chars 

In [ ]:
#semantic chunking
!pip -q install --upgrade "sentence-transformers==3.0.1" "transformers<5,>=4.41" scikit-learn nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 43.4 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
nltk.download('punkt', quiet=True)

# Load once globally
semantic_model = SentenceTransformer("all-MiniLM-L6-v2")

def semantic_chunk_text(text: str, similarity_threshold: float = 0.8, max_tokens: int = 500) -> list:
    """
    Splits text into semantic chunks based on sentence similarity and max token length.
    """
    sentences = nltk.sent_tokenize(text)
    if not sentences:
        return []

    embeddings = semantic_model.encode(sentences)

    chunks = []
    current_chunk = [sentences[0]]
    current_embedding = embeddings[0]

    for i in range(1, len(sentences)):
        sim = cosine_similarity([current_embedding], [embeddings[i]])[0][0]
        chunk_token_count = len(" ".join(current_chunk)) // 4

        if sim >= similarity_threshold and chunk_token_count < max_tokens:
            current_chunk.append(sentences[i])
            current_embedding = np.mean([current_embedding, embeddings[i]], axis=0)
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentences[i]]
            current_embedding = embeddings[i]

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


from tqdm.auto import tqdm

def semantic_chunk_pdf_pages(pages_and_texts: list,
                             similarity_threshold: float = 0.8,
                             max_tokens: int = 500) -> list[dict]:
    """
    Takes PDF pages with text and splits them into semantic chunks.

    Returns a list of dicts with page_number, chunk_index, and chunk_text.
    """
    all_chunks = []

    for page in tqdm(pages_and_texts, desc="Semantic chunking pages"):
        page_number = page["page_number"]
        page_text = page["text"]

        chunks = semantic_chunk_text(page_text,
                                     similarity_threshold=similarity_threshold,
                                     max_tokens=max_tokens)
        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "page_number": page_number,
                "chunk_index": i,
                "chunk_char_count": len(chunk),
                "chunk_word_count": len(chunk.split()),
                "chunk_token_count": len(chunk) / 4,  # rough token estimate
                "chunk_text": chunk
            })
    return all_chunks




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import nltk
nltk.download('punkt_tab')
semantic_chunked_pages = semantic_chunk_pdf_pages(pages_and_texts,
                                                  similarity_threshold=0.75,
                                                  max_tokens=500)

print(f"Total semantic chunks: {len(semantic_chunked_pages)}")
print(f"First semantic chunk (page {semantic_chunked_pages[0]['page_number']}):")
print(semantic_chunked_pages[0]['chunk_text'][:200] + "...")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Semantic chunking pages:   0%|          | 0/1208 [00:00<?, ?it/s]

Total semantic chunks: 12016
First semantic chunk (page -41):
Human Nutrition: 2020 Edition...


In [ ]:
# Pretty-print 5 random SEMANTIC chunks (uses `semantic_chunked_pages` from your code above)

import random
import textwrap

def _scattered_indices(n: int, k: int, jitter_frac: float = 0.08) -> list[int]:
    """Evenly spaced anchors + random jitter → indices scattered across [0, n-1]."""
    if k <= 0:
        return []
    if k == 1:
        return [random.randrange(n)]
    anchors = [int(round(i * (n - 1) / (k - 1))) for i in range(k)]
    out, seen = [], set()
    radius = max(1, int(n * jitter_frac))
    for a in anchors:
        lo, hi = max(0, a - radius), min(n - 1, a + radius)
        j = random.randint(lo, hi)
        if j not in seen:
            out.append(j); seen.add(j)
    while len(out) < k:
        r = random.randrange(n)
        if r not in seen:
            out.append(r); seen.add(r)
    return out

def _draw_boxed_chunk(c: dict, wrap_at: int = 96) -> str:
    approx_tokens = c.get('chunk_token_count', len(c.get('chunk_text', ''))/4)
    header = (
        f" Chunk p{c['page_number']} · idx {c['chunk_index']}  |  "
        f"chars {c['chunk_char_count']} · words {c['chunk_word_count']} · ~tokens {round(approx_tokens, 2)} "
    )
    wrapped_lines = textwrap.wrap(
        c["chunk_text"], width=wrap_at, break_long_words=False, replace_whitespace=False
    )
    content_width = max([0, *map(len, wrapped_lines)])
    box_width = max(len(header), content_width + 2)  # +2 for side padding

    top    = "╔" + "═" * box_width + "╗"
    hline  = "║" + header.ljust(box_width) + "║"
    sep    = "╟" + "─" * box_width + "╢"
    body   = "\n".join("║ " + line.ljust(box_width - 2) + " ║" for line in wrapped_lines) or \
             ("║ " + "".ljust(box_width - 2) + " ║")
    bottom = "╚" + "═" * box_width + "╝"
    return "\n".join([top, hline, sep, body, bottom])

def show_random_semantic_chunks(semantic_chunked_pages: list[dict], k: int = 5, seed: int | None = 42):
    if seed is not None:
        random.seed(seed)
    n = len(semantic_chunked_pages)
    if n == 0:
        print("No semantic chunks to display.");
        return
    idxs = _scattered_indices(n, k)
    print(f"Showing {len(idxs)} scattered random SEMANTIC chunks out of {n} total:\n")
    for i, idx in enumerate(idxs, 1):
        print(f"#{i}")
        print(_draw_boxed_chunk(semantic_chunked_pages[idx]))
        print()

# --- Run (expects you've already created `semantic_chunked_pages`) ---
assert 'semantic_chunked_pages' in globals() and len(semantic_chunked_pages) > 0, \
    "Run your semantic chunking code first to define `semantic_chunked_pages`."
show_random_semantic_chunks(semantic_chunked_pages, k=5, seed=42)


Showing 5 scattered random SEMANTIC chunks out of 12016 total:

#1
╔══════════════════════════════════════════════════════════════════════════════════════════════╗
║ Chunk p56 · idx 7  |  chars 200 · words 31 · ~tokens 50.0                                    ║
╟──────────────────────────────────────────────────────────────────────────────────────────────╢
║ Observing the connection  between the beverage and longevity, Dr. Elie Metchnikoff began his ║
║ research on beneficial bacteria and the longevity of life that led to  his book, The         ║
║ Prolongation of Life.                                                                        ║
╚══════════════════════════════════════════════════════════════════════════════════════════════╝

#2
╔════════════════════════════════════════════════════════════════════════════════════╗
║ Chunk p232 · idx 7  |  chars 82 · words 12 · ~tokens 20.5                          ║
╟───────────────────────────────────────────────────────────────────────────

In [ ]:
import nltk
from tqdm.auto import tqdm
nltk.download("punkt")

def recursive_chunk_text(text: str, max_chunk_size: int = 1000, min_chunk_size: int = 100) -> list:
    """
    Recursively splits a block of text into chunks that fit within size constraints.
    Tries splitting by sections, then newlines, then sentences.
    """
    def split_chunk(chunk: str) -> list:
        #  Base case
        if len(chunk) <= max_chunk_size:
            return [chunk]

        #  Try splitting by double newlines
        sections = chunk.split("\n\n")
        if len(sections) > 1:
            result = []
            for section in sections:
                if section.strip():
                    result.extend(split_chunk(section.strip()))
            return result

        # Try splitting by single newline
        sections = chunk.split("\n")
        if len(sections) > 1:
            result = []
            for section in sections:
                if section.strip():
                    result.extend(split_chunk(section.strip()))
            return result

        # Fallback: split by sentences
        sentences = nltk.sent_tokenize(chunk)
        chunks, current_chunk, current_size = [], [], 0

        for sentence in sentences:
            if current_size + len(sentence) > max_chunk_size:
                if current_chunk:
                    chunks.append(" ".join(current_chunk))
                current_chunk = [sentence]
                current_size = len(sentence)
            else:
                current_chunk.append(sentence)
                current_size += len(sentence)

        if current_chunk:
            chunks.append(" ".join(current_chunk))

        return chunks

    return split_chunk(text)


def recursive_chunk_pdf_pages(pages_and_texts: list,
                              max_chunk_size: int = 1000,
                              min_chunk_size: int = 100) -> list[dict]:
    """
    Takes PDF pages with text and splits them into recursive chunks.

    Returns a list of dicts with page_number, chunk_index, and chunk_text.
    """
    all_chunks = []

    for page in tqdm(pages_and_texts, desc="Recursive chunking pages"):
        page_number = page["page_number"]
        page_text = page["text"]

        chunks = recursive_chunk_text(page_text,
                                      max_chunk_size=max_chunk_size,
                                      min_chunk_size=min_chunk_size)
        for i, chunk in enumerate(chunks):
            all_chunks.append({
                "page_number": page_number,
                "chunk_index": i,
                "chunk_char_count": len(chunk),
                "chunk_word_count": len(chunk.split()),
                "chunk_token_count": len(chunk) / 4,  # rough token estimate
                "chunk_text": chunk
            })
    return all_chunks


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
recursive_chunked_pages = recursive_chunk_pdf_pages(pages_and_texts,
                                                    max_chunk_size=800,
                                                    min_chunk_size=100)

print(f"Total recursive chunks: {len(recursive_chunked_pages)}")
print(f"First recursive chunk (page {recursive_chunked_pages[0]['page_number']}):")
print(recursive_chunked_pages[0]['chunk_text'][:200] + "...")

Recursive chunking pages:   0%|          | 0/1208 [00:00<?, ?it/s]

Total recursive chunks: 2434
First recursive chunk (page -41):
Human Nutrition: 2020 Edition...


In [ ]:
# Pretty-print 5 random RECURSIVE chunks (uses `recursive_chunked_pages` from your code above)

import random
import textwrap

def _scattered_indices(n: int, k: int, jitter_frac: float = 0.08) -> list[int]:
    """Evenly spaced anchors + random jitter → indices scattered across [0, n-1]."""
    if k <= 0:
        return []
    if k == 1:
        return [random.randrange(n)]
    anchors = [int(round(i * (n - 1) / (k - 1))) for i in range(k)]
    out, seen = [], set()
    radius = max(1, int(n * jitter_frac))
    for a in anchors:
        lo, hi = max(0, a - radius), min(n - 1, a + radius)
        j = random.randint(lo, hi)
        if j not in seen:
            out.append(j); seen.add(j)
    while len(out) < k:
        r = random.randrange(n)
        if r not in seen:
            out.append(r); seen.add(r)
    return out

def _draw_boxed_chunk(c: dict, wrap_at: int = 96) -> str:
    approx_tokens = c.get('chunk_token_count', len(c.get('chunk_text', '')) / 4)
    header = (
        f" Chunk p{c['page_number']} · idx {c['chunk_index']}  |  "
        f"chars {c['chunk_char_count']} · words {c['chunk_word_count']} · ~tokens {round(approx_tokens, 2)} "
    )
    wrapped_lines = textwrap.wrap(
        c["chunk_text"], width=wrap_at, break_long_words=False, replace_whitespace=False
    )
    content_width = max([0, *map(len, wrapped_lines)])
    box_width = max(len(header), content_width + 2)  # +2 for side padding

    top    = "╔" + "═" * box_width + "╗"
    hline  = "║" + header.ljust(box_width) + "║"
    sep    = "╟" + "─" * box_width + "╢"
    body   = "\n".join("║ " + line.ljust(box_width - 2) + " ║" for line in wrapped_lines) or \
             ("║ " + "".ljust(box_width - 2) + " ║")
    bottom = "╚" + "═" * box_width + "╝"
    return "\n".join([top, hline, sep, body, bottom])

def show_random_recursive_chunks(recursive_chunked_pages: list[dict], k: int = 5, seed: int | None = 42):
    if seed is not None:
        random.seed(seed)
    n = len(recursive_chunked_pages)
    assert n > 0, "No recursive chunks to display. Did you run the recursive chunking cell?"
    idxs = _scattered_indices(n, k)
    print(f"Showing {len(idxs)} scattered random RECURSIVE chunks out of {n} total:\n")
    for i, idx in enumerate(idxs, 1):
        print(f"#{i}")
        print(_draw_boxed_chunk(recursive_chunked_pages[idx]))
        print()

# --- Run (expects you've already created `recursive_chunked_pages`) ---
assert 'recursive_chunked_pages' in globals() and len(recursive_chunked_pages) > 0, \
    "Run your recursive chunking code first to define `recursive_chunked_pages`."
show_random_recursive_chunks(recursive_chunked_pages, k=5, seed=42)


Showing 5 scattered random RECURSIVE chunks out of 2434 total:

#1
╔═══════════════════════════════════════════════════════════════════════╗
║ Chunk p53 · idx 0  |  chars 69 · words 14 · ~tokens 17.25             ║
╟───────────────────────────────────────────────────────────────────────╢
║ PART II  CHAPTER 2. THE HUMAN  BODY  Chapter 2. The Human Body  |  53 ║
╚═══════════════════════════════════════════════════════════════════════╝

#2
╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║ Chunk p213 · idx 0  |  chars 761 · words 102 · ~tokens 190.25                                    ║
╟──────────────────────────────────────────────────────────────────────────────────────────────────╢
║ Deadly  water-borne  illnesses  decreased to  almost  nonexistent  levels in the  United  States ║
║ after  the  implementat ion of water  disinfection  methods. public water systems in the country ║
║ adhere to the standards. About  15 percent of Americ

In [ ]:
import re
import random
import textwrap

# 1) Helper to detect "chapter start" pages
def _is_chapter_header_page(text: str) -> bool:
    # Robust to punctuation/diacritics differences; matches the recurring header
    # e.g., "UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM"
    return re.search(r"university\s+of\s+hawai", text, flags=re.IGNORECASE) is not None

def _guess_title_from_page(text: str) -> str:
    """
    Best-effort chapter title guess = the text before the 'University of Hawai' header line.
    Falls back to the first ~120 characters.
    """
    m = re.search(r"university\s+of\s+hawai", text, flags=re.IGNORECASE)
    if m:
        title = text[:m.start()].strip()
        # keep it readable
        title = re.sub(r"\s+", " ", title).strip()
        if 10 <= len(title) <= 180:
            return title
    # fallback
    t = re.sub(r"\s+", " ", text).strip()
    return t[:120] if t else "Untitled Chapter"

# 2) Build chapter chunks
def chapter_chunk_pdf_pages(pages_and_texts: list[dict]) -> list[dict]:
    """
    Returns a list of chapter chunks:
    [
      {
        'chapter_index': int,
        'title': str,
        'page_start': int,   # adjusted page number (your -41 offset)
        'page_end': int,
        'chunk_char_count': int,
        'chunk_word_count': int,
        'chunk_token_count': float,   # ~chars/4
        'chunk_text': str
      }, ...
    ]
    """
    if not pages_and_texts:
        return []

    # Find all page indices that look like the start of a chapter
    chapter_starts = []
    for i, p in enumerate(pages_and_texts):
        txt = p["text"]
        if _is_chapter_header_page(txt):
            chapter_starts.append(i)

    # If nothing detected, return empty (or treat entire doc as one chunk)
    if not chapter_starts:
        # Treat entire doc as one "chapter"
        all_text = " ".join(p["text"] for p in pages_and_texts).strip()
        return [{
            "chapter_index": 0,
            "title": _guess_title_from_page(pages_and_texts[0]["text"]),
            "page_start": pages_and_texts[0]["page_number"],
            "page_end": pages_and_texts[-1]["page_number"],
            "chunk_char_count": len(all_text),
            "chunk_word_count": len(all_text.split()),
            "chunk_token_count": round(len(all_text) / 4, 2),
            "chunk_text": all_text
        }]

    # Build chapter ranges (start -> next_start-1)
    chapter_chunks = []
    for ci, s in enumerate(chapter_starts):
        e = (chapter_starts[ci + 1] - 1) if (ci + 1 < len(chapter_starts)) else (len(pages_and_texts) - 1)
        if e < s:
            continue  # guard (shouldn't happen)

        pages = pages_and_texts[s:e + 1]
        text_concat = " ".join(p["text"] for p in pages).strip()
        title = _guess_title_from_page(pages[0]["text"])

        chapter_chunks.append({
            "chapter_index": ci,
            "title": title,
            "page_start": pages[0]["page_number"],
            "page_end": pages[-1]["page_number"],
            "chunk_char_count": len(text_concat),
            "chunk_word_count": len(text_concat.split()),
            "chunk_token_count": round(len(text_concat) / 4, 2),
            "chunk_text": text_concat
        })

    return chapter_chunks

In [ ]:
structure_chunked_pages = chapter_chunk_pdf_pages(pages_and_texts)

print(f"Total chapter-based chunks: {len(structure_chunked_pages)}")
if structure_chunked_pages:
    first = structure_chunked_pages[0]
    print(f"First chapter (pages {first['page_start']}–{first['page_end']}): {first['title']}")
    print(first['chunk_text'][:200] + "...")
else:
    print("No chapters detected.")

Total chapter-based chunks: 171
First chapter (pages -39–-39): Human Nutrition: 2020 Edition
Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  K...


In [ ]:
try:
    from ipywidgets import Widget
    # This closes all active widget instances (like progress bars)
    # effectively clearing the "state" metadata causing the error.
    Widget.close_all()
    print("Success: All widget states have been cleared.")
except ImportError:
    print("No widgets library found, so no widgets to clear.")
except Exception as e:
    print(f"An error occurred: {e}")

Success: All widget states have been cleared.
